# Predict Mitosis Cells

In [ ]:
# from livecell_tracker.sample_data import tutorial_three_image_sys
from pathlib import Path
from livecell_tracker.core.datasets import LiveCellImageDataset
import glob

dataset_dir_path = Path(
    "../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/XY16/"
)

mask_dataset_path = Path("../datasets/EBSS_Starvation/tif_STAV-A549_VIM_24hours_NoTreat_NA_YL_Ti2e_2022-12-21/out/XY16/seg")

mask_dataset = LiveCellImageDataset(mask_dataset_path, ext="png")
time2url = sorted(glob.glob(str((Path(dataset_dir_path) / Path("*_DIC.tif")))))
time2url = {i: path for i, path in enumerate(time2url)}
dic_dataset = LiveCellImageDataset(time2url=time2url, ext="tif")

In [ ]:
from livecell_tracker.core.io_sc import prep_scs_from_mask_dataset
scs = prep_scs_from_mask_dataset(mask_dataset, dic_dataset)

In [ ]:
from typing import List
from livecell_tracker.track.sort_tracker_utils import (
    gen_SORT_detections_input_from_contours,
    update_traj_collection_by_SORT_tracker_detection,
    track_SORT_bbox_from_contours,
    track_SORT_bbox_from_scs
)


traj_collection = track_SORT_bbox_from_scs(scs, dic_dataset, mask_dataset=mask_dataset, max_age=1, min_hits=1)

## Load a model to predict mitosis and apoptosis 

In [ ]:
from livecell_tracker.classification.cnn_mitosis_apoptosis import MitApopImageClassifier, MitApopImageDataset
import torch
from PIL import Image
import torchvision.transforms as transforms


In [ ]:

model = MitApopImageClassifier.load_from_checkpoint("./sc_classify_MA/version_15/checkpoints/epoch=8932-step=812902.ckpt", dir_path=None)

transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.5, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    # transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
])

model = model.eval()

In [ ]:
from matplotlib import pyplot as plt
from livecell_tracker.preprocess.utils import normalize_img_to_uint8
import numpy as np

model = model.eval()

padding = 20
for sc in scs:
# for _ in range(1000):
    sc = scs[np.random.randint(0, len(scs))]
    img = sc.get_img_crop(padding=padding)
    img = normalize_img_to_uint8(img)
    img = Image.fromarray(img)
    img = transform(img)
    img = torch.stack([torch.tensor(img), torch.tensor(img), torch.tensor(img)])
    img = img.squeeze()
    # print("img shape:", img.shape)
    img = img.unsqueeze(0)
    pred = model(img)
    probs = torch.softmax(pred, dim=1)
    
    if probs[0][1] > 0.5:
        print(probs)
        orig_img = sc.get_img_crop(padding=padding)
        plt.imshow(orig_img)
        plt.title(f"pred: {list(probs.cpu().detach().numpy())}")
        plt.show()
    # break